# Download Twitter data using API Keys
All codes inspired by Vicky Qian: https://gist.github.com/vickyqian

In [ ]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open('g.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="global warming",count=100000,
                           lang="en",
                           since="2008-01-01").items():
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [ ]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

In [ ]:
df = pd.read_csv("gb.csv",header=None)

In [ ]:
df.rename(columns={0:"Time", 1:"Content"}, inplace=True)
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
import csv

def unicode_csv_reader(unicode_csv_data, dialect=csv.excel, **kwargs):
    # csv.py doesn't do Unicode; encode temporarily as UTF-8:
    csv_reader = csv.reader(utf_8_encoder(unicode_csv_data),
                            dialect=dialect, **kwargs)
    for row in csv_reader:
        # decode UTF-8 back to Unicode, cell by cell:
        yield [unicode(cell, 'utf-8') for cell in row]

def utf_8_encoder(unicode_csv_data):
    for line in unicode_csv_data:
        yield line.encode('utf-8')

In [ ]:
train = pd.read_csv("global_warming_train.csv", encoding = 'unicode_escape')

In [ ]:
train.shape

In [ ]:
###Preprocess tweets
def processTweet2(tweet):
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet    

In [ ]:
###get stopword list
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

In [ ]:
stopWords = []

st = open('stopword.txt', 'r')
stopWords = getStopWordList('stopword.txt')


def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [ ]:
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [ ]:
import re
re.compile('<title>(.*)</title>')

In [ ]:
###load gb sentiment training data 
    
gbtrain = pd.read_csv("global_warming_train.csv", encoding ="ISO-8859-1")
tweets = []
featureList = []
for i in range(len(gbtrain)):
    sentiment = gbtrain['existence'][i]
    tweet = gbtrain['tweet'][i]
    processedTweet = processTweet2(tweet)
    featureVector = getFeatureVector(processedTweet)
    featureList.extend(featureVector)
    tweets.append((featureVector, sentiment))

In [ ]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [ ]:
### Remove featureList duplicates
featureList = list(set(featureList))

In [ ]:
import nltk
training_set = nltk.classify.util.apply_features(extract_features, tweets)
# Train the classifier Naive Bayes Classifier
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)
#df is a dataframe containing all the tweets
df['sentiment'] = df['Content'].apply(lambda tweet: NBClassifier.classify(extract_features(getFeatureVector(processTweet2(tweet)))))

In [ ]:
df

In [ ]:
df.to_csv("gb_prediction.csv")

In [2]:
import pandas as pd

In [6]:
prediction = pd.read_csv("predition_df.csv")

In [11]:
prediction[prediction.sentiment == 'Y'].count()

Unnamed: 0    88988
Time          88988
Content       88988
sentiment     88988
dtype: int64

In [14]:
prediction.drop(columns = {"Unnamed: 0"}, inplace=True)

In [19]:
prediction.isna().sum()

Time            0
Content         0
sentiment    8033
dtype: int64